In [31]:
import tensorflow as tf
import base64
import json
import random
import pandas as pd
import requests

In [32]:
df = pd.read_csv('data/output.csv')

df.head(5)

,Number,Sex,Red pixel,Green pixel,Blue pixel,Hb,Anaemic
0,1,M,43.2555,30.8421,25.9025,6.3,Yes
1,2,F,45.6033,28.1900,26.2067,13.5,No
2,3,F,45.0107,28.9677,26.0215,11.7,No
3,4,F,44.5398,28.9899,26.4703,13.5,No
4,5,M,43.2870,30.6972,26.0158,12.4,No


In [35]:
df = pd.read_csv("data/output.csv")
y = df.pop("Anaemic")

columns = df.columns.values
rand = random.randint(0, len(df))

features = df.values[rand]
label = y[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Number': 25,
 'Sex': 'M',
 'Red pixel': 45.6906,
 'Green pixel': 30.0735,
 'Blue pixel': 24.2359,
 'Hb': 9.1}

In [36]:
# Functions for creating features
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

In [37]:
# Modify prepare_json to handle string columns properly
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):  # Check if it's a string (e.g., 'Sex' column)
            feature_spec[keys] = string_feature(values.strip())  # Strip any extra spaces
        elif keys == "wtkg":  # Handle special case for weight (for example)
            feature_spec[keys] = float_feature(float(values))
        elif keys == "Number":  # Make sure 'Number' is treated as integer
            feature_spec[keys] = int_feature(int(float(values)))  # Convert to integer
        else:
            feature_spec[keys] = float_feature(float(values))  # Handle numerical features

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

In [38]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = (
        "https://anemia-detection.up.railway.app/v1/models/anemia-model:predict"
    )
    response = requests.post(endpoint, data=json_data)
    
    # Cetak respons JSON untuk memeriksa struktur
    print(response.json())

    # Periksa apakah respons mengandung kunci 'predictions'
    if 'predictions' in response.json():
        prediction = response.json()["predictions"][0][0]
        if prediction < 0.6:
            return "Not Anemia"
        else:
            return "Anemia"
    else:
        return "Error: 'predictions' key not found in response."

In [39]:
make_predictions(inputs)

{'predictions': [[0.00468569249]]}


'Not Anemia'